In [1]:
import numpy as np
import pandas as pd
from caveclient import CAVEclient
from run_skel_task import write_template

client = CAVEclient(
    'v1dd',
)

# client = CAVEclient(
    # 'minnie65_public',
# )

data_dir = f"data/{client.datastack_name}"
skel_dir = f"skeletons/{client.datastack_name}"

import os
if not os.path.exists(skel_dir):
    os.makedirs(skel_dir)
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

#### Use nucleus IDs to generate the dataframe
* Update the nucleus I
* Change the soma radius as needed

In [2]:
import pathlib

In [3]:
pathlib.Path(skel_dir).resolve()

PosixPath('/Users/caseysm/Work/Code/Tasks/skeletonization/skeletons/v1dd')

In [4]:
df = pd.read_feather('/Users/caseysm/Work/Projects/layer1/l1_pf_df.feather')

In [7]:
ts = client.materialize.get_timestamp()

In [15]:
from standard_transform.datasets import v1dd_ds 

pf_df = client.materialize.tables.ariadne_axon_task().query(desired_resolution=[1,1,1])
pf_df = pf_df.drop_duplicates('pt_root_id', keep='first')
pf_df['pt_x'] = v1dd_ds.transform_nm.apply_dataframe('pt_position', pf_df, projection='x')
pf_df['pt_y'] = v1dd_ds.transform_nm.apply_dataframe('pt_position', pf_df, projection='y')
pf_df['pt_z'] = v1dd_ds.transform_nm.apply_dataframe('pt_position', pf_df, projection='z')

drop_idx = pf_df.query('cell_type in ["withdrawn", "given", "error", "initial_progress"]').index
pf_df = pf_df.drop(drop_idx)

bad_entries = pf_df.query('pt_y < -100').index
pf_df = pf_df.drop(bad_entries)

In [11]:
import json
with open("/Users/caseysm/Work/Projects/MinnieColumn/pipelines/main/skeleton_radius.json", 'r') as f:
    soma_radius = json.load(f)

In [24]:
radius_df = pd.DataFrame(
    {
        'id': [int(x) for x in soma_radius.keys()],
        'soma_radius': list(soma_radius.values()),
    }
)

In [26]:
df = df.merge(
    radius_df,
    on='id',
)

In [61]:
# soma_df = client.materialize.tables.nucleus_neuron_svm(
#     pt_root_id=root_ids, cell_type="neuron"
# ).query(desired_resolution=[1, 1, 1], timestamp=ts).drop_duplicates('pt_root_id')

# job_df = soma_df.reset_index()

Table Owner Notice on nucleus_neuron_svm: Please cite https://doi.org/10.1101/2022.07.20.499976 when using this table.


In [20]:
job_name = f'proofread_v{client.materialize.version}'
soma_radius = 12_500

job_df = pf_df.query('cell_type in ["complete", "submitted", "clean"]').reset_index(drop=True)
job_df['soma_radius'] = soma_radius

data_dir = f"data/{client.datastack_name}"
skel_dir = f"skeletons/{client.datastack_name}"

write_template(
    f'{job_name}.env',
    job_df,
    f"{data_dir}/{job_name}.feather",
    skel_dir,
    client,
    collapse_soma=True,
    root_id_column='pt_root_id',
    soma_radius_column='soma_radius',
    parallel=8,
    split_threshold=0.45,
    timestamp=ts.timestamp(),
)
print(f'To run: python run_skel_task.py -c {job_name}.env')

Wrote template:
            
    FILEPATH=file://skeletons/v1dd
    DATASTACK=v1dd
    SERVER_ADDRESS=https://globalv1.em.brain.allentech.org
    TIMESTAMP=1717488601.302705
    COLLAPSE_SOMA=True
    DATAFRAME=data/v1dd/proofread_v757.feather
    ROOT_ID_COLUMN=pt_root_id
    SOMA_POINT_COLUMN=pt_position
    SOMA_RADIUS_COLUMN=soma_radius
    PARALLEL=8
    SPLIT_THRESHOLD=0.45
    
            
To run: python run_skel_task.py -c proofread_v757.env


In [5]:
df

,id,created,superceded_id,valid,classification_system,cell_type,pt_supervoxel_id,pt_root_id,pt_position,soma_id,soma_depth
0,128,2022-03-28 12:58:22.065467+00:00,NaN,t,vip_task,submitted,91838908337575741,864691132686850954,"[788220.0, 424287.0, 220500.0]",265430,78.630400
1,225,2022-03-28 13:20:21.781399+00:00,NaN,t,aom_task,submitted,84590859169177094,864691132936402059,"[531666.0, 421515.0, 292275.0]",113712,96.449689
2,211,2022-03-28 13:19:21.096148+00:00,NaN,t,brm_task,submitted,92470990084783504,864691132771211668,"[809541.0, 379818.0, 227745.0]",265452,37.618042
3,617,2022-04-14 13:27:05.011389+00:00,NaN,t,brm_task,submitted,93880426552724496,864691132586460898,"[859257.0, 454500.0, 228150.0]",265472,109.434626
4,619,2022-04-14 13:27:05.037600+00:00,NaN,t,brm_task,submitted,93457939343981680,864691132767824033,"[845838.0, 443556.0, 276300.0]",266038,111.845956
...,...,...,...,...,...,...,...,...,...,...,...
70,3210,2022-09-16 00:53:52.111125+00:00,NaN,t,emct_task,submitted,92121070576200763,864691132684814532,"[796635.0, 450234.0, 243585.0]",265552,109.767783
71,3211,2022-09-16 00:53:52.125583+00:00,NaN,t,emct_task,submitted,95076489179119662,864691132591555325,"[902178.0, 446814.0, 259515.0]",340348,110.211493
72,3659,2023-06-01 22:57:03.160170+00:00,NaN,t,emct_task,submitted,93950383315486108,864691132716483917,"[863298.0, 438435.0, 339885.0]",266801,123.845716
73,3422,2022-11-22 19:29:40.209871+00:00,NaN,t,emct_task,submitted,91417383134364523,864691132655360252,"[773586.0, 448992.0, 241920.0]",265541,108.240159


In [9]:
job_df = df.copy()
job_name = 'l1_neurons'
soma_radius = 12_500
job_df['soma_radius'] = soma_radius

data_dir = f"data/{client.datastack_name}"
skel_dir = f"skeletons/{client.datastack_name}"

write_template(
    f'{job_name}.env',
    job_df,
    f"{data_dir}/{job_name}.feather",
    skel_dir,
    client,
    collapse_soma=True,
    root_id_column='pt_root_id',
    soma_radius_column='soma_radius',
    parallel=8,
    split_threshold=0.35,
    timestamp=ts.timestamp(),
)
print(f'To run: python run_skel_task.py -c {job_name}.env')

Wrote template:
            
    FILEPATH=file://skeletons/v1dd
    DATASTACK=v1dd
    SERVER_ADDRESS=https://globalv1.em.brain.allentech.org
    TIMESTAMP=1721635801.30137
    COLLAPSE_SOMA=True
    DATAFRAME=data/v1dd/l1_neurons.feather
    ROOT_ID_COLUMN=pt_root_id
    SOMA_POINT_COLUMN=pt_position
    SOMA_RADIUS_COLUMN=soma_radius
    PARALLEL=8
    SPLIT_THRESHOLD=0.35
    
            
To run: python run_skel_task.py -c l1_neurons.env


In [10]:
soma

,id,created,valid,volume,pt_supervoxel_id,pt_root_id,id_ref,created_ref,valid_ref,target_id,classification_system,cell_type,pt_position,bb_start_position,bb_end_position
0,258319,2020-09-28 22:40:42.476911+00:00,t,261.806162,89309001002848425,864691137054893686,99,2023-08-21 22:53:34.255741+00:00,t,258319,excitatory,L2c,"[713600.0, 572992.0, 849520.0]","[nan, nan, nan]","[nan, nan, nan]"
1,276438,2020-09-28 22:40:42.700226+00:00,t,277.317714,89465269428261699,864691136487559186,1160,2023-08-21 22:53:35.075622+00:00,t,276438,excitatory,L6tall-b,"[718592.0, 1035072.0, 943880.0]","[nan, nan, nan]","[nan, nan, nan]"
2,260552,2020-09-28 22:40:42.745779+00:00,t,230.111805,89170256379033022,864691136040432126,100,2023-08-21 22:53:34.256631+00:00,t,260552,excitatory,L4a,"[709632.0, 631872.0, 840080.0]","[nan, nan, nan]","[nan, nan, nan]"
3,260263,2020-09-28 22:40:42.746658+00:00,t,274.324193,88044356338331571,864691136334881203,101,2023-08-21 22:53:34.257543+00:00,t,260263,excitatory,L3b,"[677760.0, 632512.0, 810640.0]","[nan, nan, nan]","[nan, nan, nan]"
4,262898,2020-09-28 22:40:42.749245+00:00,t,230.092308,88468836747612860,864691135654475970,1405,2023-08-21 22:53:35.262828+00:00,t,262898,inhibitory,ITC,"[690048.0, 701120.0, 878560.0]","[nan, nan, nan]","[nan, nan, nan]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1346,267029,2020-09-28 22:45:25.458678+00:00,t,540.028928,88543328459028709,864691136137767293,1263,2023-08-21 22:53:35.153486+00:00,t,267029,excitatory,L5ET,"[692032.0, 823424.0, 815640.0]","[nan, nan, nan]","[nan, nan, nan]"
1347,267294,2020-09-28 22:45:25.482981+00:00,t,550.195200,88894897570199057,864691135182879746,1233,2023-08-21 22:53:35.129406+00:00,t,267294,excitatory,L5ET,"[701504.0, 814912.0, 889560.0]","[nan, nan, nan]","[nan, nan, nan]"
1348,303145,2020-09-28 22:45:25.534382+00:00,t,570.206290,90302616051180901,864691135688186080,1235,2023-08-21 22:53:35.130803+00:00,t,303145,excitatory,L5ET,"[742528.0, 824896.0, 890640.0]","[nan, nan, nan]","[nan, nan, nan]"
1349,302955,2020-09-28 22:45:25.587619+00:00,t,612.866294,89949329020077684,864691135851941575,1225,2023-08-21 22:53:35.123719+00:00,t,302955,excitatory,L5ET,"[731456.0, 781760.0, 832680.0]","[nan, nan, nan]","[nan, nan, nan]"


In [53]:
job_df = client.materialize.tables.l5et_column().query(desired_resolution=[1,1,1])

job_name = 'l5et_cells_to_viz'
soma_radius = 12_500
job_df['soma_radius'] = soma_radius

data_dir = f"data/{client.datastack_name}"
skel_dir = f"skeletons/{client.datastack_name}"

write_template(
    f'{job_name}.env',
    job_df,
    f"{data_dir}/{job_name}.feather",
    skel_dir,
    client,
    collapse_soma=True,
    root_id_column='pt_root_id',
    soma_radius_column='soma_radius',
    parallel=8,
    split_threshold=0.05,
    timestamp=ts.timestamp(),
)
print(f'To run: python run_skel_task.py -c {job_name}.env')

Wrote template:
            
    FILEPATH=file://skeletons/minnie65_phase3_v1
    DATASTACK=minnie65_phase3_v1
    SERVER_ADDRESS=https://global.daf-apis.com
    TIMESTAMP=1688112601.220633
    COLLAPSE_SOMA=True
    DATAFRAME=data/minnie65_phase3_v1/l5et_cells_to_viz.feather
    ROOT_ID_COLUMN=pt_root_id
    SOMA_POINT_COLUMN=pt_position
    SOMA_RADIUS_COLUMN=soma_radius
    PARALLEL=8
    SPLIT_THRESHOLD=0.05
    
            
To run: python run_skel_task.py -c l5et_cells_to_viz.env


In [ ]:
data = [['SA198', 307168],
         ['VA78', 371370],
         ['AA115', 277304],
         ['AA116', 277367],
         ['BA135', 397422],
         ['BA136', 396903],
         ['CA292', 352203],
         ['CA322', 370537],
         ['SA187', 382466],
         ['SA189', 309307],
         ['AA114', 277114],
         ['CA463', 265358],
         ['VA74', 366888],
         ['VA76', 281467],
         ['VA79', 370920],
         ['CA417', 385831],
         ['CA398', 294794],
         ['VA86', 355209],
         ['VA82', 370310],
         ['VA84', 371716],
         ['SA203', 296128],
         ['CA109', 383747],
         ['SA199', 371111],
         ['SA201', 281103]]

# Create a DataFrame from the list of lists
pf_df = pd.DataFrame(data, columns=['task_no', 'nucleus_id'])

In [ ]:
job_df = client.materialize.tables.nucleus_detection_v0(id=pf_df['nucleus_id']).query(desired_resolution=[1,1,1])

job_name = 'ariadne_check_may'
soma_radius = 12_500
job_df['soma_radius'] = soma_radius

data_dir = f"data/{client.datastack_name}"
skel_dir = f"skeletons/{client.datastack_name}"

write_template(
    f'{job_name}.env',
    job_df,
    f"{data_dir}/{job_name}.feather",
    skel_dir,
    client,
    collapse_soma=True,
    root_id_column='pt_root_id',
    soma_radius_column='soma_radius',
    parallel=8,
    split_threshold=0,
)
print(f'To run: python run_skel_task.py -c {job_name}.env')

In [ ]:
from standard_transform import v1dd_ds

nuc_df = client.materialize.tables.nucleus_detection_v0().query(desired_resolution=[9,9,45])
nuc_df.drop_duplicates('pt_root_id', keep=False, inplace=True)

base_df = client.materialize.tables.manual_central_types().query(desired_resolution=[9,9,45])

pts_t = v1dd_ds.transform_nm.apply_dataframe('pt_position', base_df)

xyz0 = v1dd_ds.transform_vx.invert(v1dd_ds.streamline_nm._points[10])

xyz1 = np.vstack(base_df['pt_position'])

xyz1 = np.vstack(nuc_df['pt_position'])

ds = v1dd_ds.streamline_vx.radial_distance(
    xyz0,
    xyz1,
    return_angle=False,
)

In [ ]:
run_ids = nuc_df[ds<175].query('volume > 218')['id'].values

In [ ]:
job_df = client.materialize.tables.nucleus_detection_v0(id=run_ids).query(desired_resolution=[1,1,1])

job_name = 'v1dd_single_soma_r3'
soma_radius = 12_500
job_df['soma_radius'] = soma_radius

data_dir = f"data/{client.datastack_name}"
skel_dir = f"skeletons/{client.datastack_name}"

write_template(
    f'{job_name}.env',
    job_df,
    f"{data_dir}/{job_name}.feather",
    skel_dir,
    client,
    collapse_soma=True,
    root_id_column='pt_root_id',
    soma_radius_column='soma_radius',
    parallel=8,
    split_threshold=0,
)

In [ ]:
print(f'To run: python run_skel_task.py -c {job_name}.env')

In [ ]:
client.chunkedgraph.level2_chunk_graph(864691129135208012)

In [ ]:
v1dd_ds.transform_nm.invert(np.median(pts_t, axis=0)) / [9, 9, 45]

In [ ]:
cell_df = client.materialize.query_table(
    "ariadne_axon_task", desired_resolution=[1, 1, 1]
)

cell_df.query('cell_type in ["submitted", "complete"]')[
    ["id", "pt_root_id", "pt_position"]
]

nuc_df = client.materialize.query_table(
    "nucleus_detection_v0", filter_in_dict={"pt_root_id": pf_ids}
).drop_duplicates("pt_root_id", keep=False)

job_df = cell_df.query('cell_type in ["submitted", "complete"]')[
    ["pt_root_id", "pt_position"]
].merge(
    nuc_df[["pt_root_id", "id"]],
    on="pt_root_id",
    how="inner",
).reset_index(drop=True)


job_name = 'v1dd_proofread_axon'
soma_radius = 10_000
job_df['soma_radius'] = soma_radius

data_dir = f"data/{client.datastack_name}"
skel_dir = f"skeletons/{client.datastack_name}"

write_template(
    f'{job_name}.env',
    job_df,
    f"{data_dir}/{job_name}.feather",
    skel_dir,
    client,
    collapse_soma=True,
    root_id_column='pt_root_id',
    soma_radius_column='soma_radius',
    parallel=8,
    split_threshold=0,
)

In [ ]:
client.materialize.version = 661
job_name = 'v1dd_proofread_axon'
soma_radius = 10_000
job_df = df[['id', 'pt_root_id', 'pt_position']].reset_index(drop=True)
job_df['soma_radius'] = soma_radius

In [ ]:
client.materialize.version = 661
job_name = 'minnie_inhib'
soma_radius = 10_000
df = pd.read_feather('../../../Projects/minnie/pf_inh_df.feather')

job_df = df[['id', 'pt_root_id', 'pt_position']].reset_index(drop=True)
job_df['soma_radius'] = soma_radius

In [ ]:
job_df

In [ ]:
client.materialize.tables

In [ ]:
job_name = 'l5et_cells'
soma_radius = 15_000
# pf_root_ids = np.unique(np.load('../../../Projects/minnie/pf_syn_ids.npy'))

df = client.materialize.tables.allen_column_mtypes_v1(cell_type='L5ET').query(desired_resolution=[1,1,1])
job_df = df[['id', 'pt_root_id', 'pt_position']].reset_index(drop=True)
job_df['soma_radius'] = soma_radius

In [ ]:
job_df

job_name = 'all_proofread_neurons'
soma_radius = 10_000

df = client.materialize.query_table(
    'ariadne_axon_task',
    desired_resolution=[1,1,1],
    filter_in_dict={'cell_type': ['submitted', 'complete']}
)
df.drop_duplicates('pt_root_id', inplace=True)
df = df.query('pt_root_id != 0').reset_index(drop=True)

job_df = df[['id', 'pt_root_id', 'pt_position']].reset_index(drop=True)
job_df['soma_radius'] = soma_radius
soma_df = client.materialize.query_table(
    'nucleus_detection_v0',
    filter_in_dict={'pt_root_id': job_df['pt_root_id'].values},
    desired_resolution=[1,1,1],
)
multi_ids = soma_df.groupby('pt_root_id').count().query('id>1').index.values
job_df = job_df.query('pt_root_id not in @multi_ids').reset_index(drop=True)

In [ ]:
from cloudfiles import CloudFiles
cf = CloudFiles(f'file://{skel_dir}')

In [ ]:
write_template(
    f'{job_name}.env',
    job_df,
    f"{data_dir}/{job_name}.feather",
    skel_dir,
    client,
    collapse_soma=True,
    root_id_column='pt_root_id',
    soma_radius_column='soma_radius',
    parallel=8,
    split_threshold=0,
)

print(f'To run: python run_skel_task.py -c {job_name}.env')

In [ ]:
client.materialize.get_timestamp(version=661).timestamp()

In [ ]:
nrn_test = basic_skeleton(
    864691135857324590,
    client,
    soma_point = [694976.0, 604608.0, 890160.0],
    soma_radius = 10000,
    timestamp=client.materialize.get_timestamp(version=661).timestamp()
)

In [ ]:
scaling = np.array(client.chunkedgraph.segmentation_info['scales'][0]['resolution']) / client.info.viewer_resolution()

ctr = (
    np.array(client.chunkedgraph.segmentation_info["scales"][0]["voxel_offset"])
    + np.array(client.chunkedgraph.segmentation_info["scales"][0]["size"])
    / 2
) * scaling

In [ ]:
f"{0.000000001:1e}"

In [ ]:
ctr

In [ ]:
client.

In [ ]:
np.array(client.chunkedgraph.segmentation_info['scales'][0]['voxel_offset'])

In [ ]:
client.chunkedgraph.segmentation_info['scales'][0]['resolution']

In [ ]:
nrn_test.anno.pre_syn.df

In [ ]:
client.materialize.get_table_metadata('coregistration_manual')